In [1]:
#import sys
#!{sys.executable} -m pip install faiss-cpu --no-cache

In [ ]:
import numpy as np
import faiss


index = faiss.IndexFlatL2(2048)
descriptors = np.vstack(descriptors)
index.add(descriptors)

In [1]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import pandas as pd
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data_utils
import numpy as np

from torch.autograd import Variable
from sklearn.model_selection import train_test_split
from sklearn import datasets
from torch.optim.lr_scheduler import StepLR

In [2]:
class IrisNet(nn.Module):
    
    def __init__(self, input_size, hidden_size1, hidden_size2, num_classes):
        
        super(IrisNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.relu1 = nn.ReLU()
        self.fc3 = nn.Linear(hidden_size2, num_classes)
        
    def forward(self, x):
    
        out = self.fc1(x)
        out = self.relu1(x)
        out = self.fc2(out)
        out = self.relu2(out)
        out = self.fc3(out)
        
        return out

In [3]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            #if args.dry_run:
             #   break

In [4]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [7]:
iris = datasets.load_iris()
df = pd.read_csv(r'data/iris.csv')
df.columns = ['sepal_length','sepal_width','petal_length','petal_width','class']
df['class'] = pd.factorize(df['class'])[0]
train, test = train_test_split(df, test_size=0.2)

In [8]:
train_target = torch.tensor(train.iloc[:,-1].values.astype(np.float32))
train = torch.tensor(train.drop('class', axis = 1).values.astype(np.float32))

test_target = torch.tensor(test.iloc[:,-1].values.astype(np.float32))
test = torch.tensor(test.drop('class', axis = 1).values.astype(np.float32))

In [9]:
train_tensor = data_utils.TensorDataset(train, train_target) 
test_tensor = data_utils.TensorDataset(test, test_target)

In [10]:
train_loader = torch.utils.data.DataLoader(dataset=train_tensor, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_tensor, batch_size=batch_size, shuffle=True)

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
batch_size = 25
model = IrisNet(4, 50, 70, 3).to(device)
optimizer = optim.Adadelta(model.parameters(), lr=0.005)

scheduler = StepLR(optimizer, step_size=1, gamma=0.2)
for epoch in range(1, 100):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
    scheduler.step()

save_model = None
    
if save_model:
    torch.save(model.state_dict(), "mnist_cnn.pt")